In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Reading Berlin Population data

In [23]:
url = r'https://de.wikipedia.org/wiki/Verwaltungsgliederung_Berlins'
tables = pd.read_html(url) # Returns list of all tables on page
population = tables[1] # Select table of interest
population.tail(100)

,Nr.,Ortsteil,Bezirk,Fläche(km²),Einwohner[2](31. Dezember 2019),Einwohnerpro km²
0,101,Mitte,Mitte,1070,102.465,9576.000
1,102,Moabit,Mitte,772,80.495,10.427
2,103,Hansaviertel,Mitte,53,5.926,11.181
3,104,Tiergarten,Mitte,517,14.881,2878.000
4,105,Wedding,Mitte,923,86.806,9405.000
5,106,Gesundbrunnen,Mitte,613,95.175,15.526
6,201,Friedrichshain,Friedrichshain-Kreuzberg,978,136.035,13.910
7,202,Kreuzberg,Friedrichshain-Kreuzberg,1040,154.351,14.841
8,301,Prenzlauer Berg,Pankow,1100,165.003,15.000
9,302,Weißensee,Pankow,793,54.032,6814.000


## Reading Berlin Crimes data

In [104]:
import pandas as pd
crimes = pd.read_csv("Berlin_crimes.csv")

# take the crime on year 2019
crimes2019 = crimes.loc[crimes['Year'] == 2019]
crimes2019.shape

(150, 20)

## Procesing the Data

### add density to crime DF

In [131]:
crimes2019_density = crimes2019

for i, c in crimes2019.iterrows():
    normalizedCode = str(c['Code'])[:-2]
    density = np.NaN
    if population.loc[population['Nr.'] == int(normalizedCode)].shape[0] != 0:
        density = population.loc[population['Nr.'] == int(normalizedCode)]['Einwohnerpro km²'].item()   
    crimes2019_density.at[i,'Density'] = density
    
print(crimes2019_density.shape)
crimes2019_density.head()

(150, 21)


,Year,District,Code,Location,Robbery,Street_robbery,Injury,Agg_assault,Threat,Theft,Car,From_car,Bike,Burglary,Fire,Arson,Damage,Graffiti,Drugs,Local,Density
1050,2019,Mitte,10111,Tiergarten Süd,60,35,365,92,128,2271,15,198,296,55,13,6,347,77,231,980,9576.000
1051,2019,Mitte,10112,Regierungsviertel,42,20,554,136,152,3692,13,172,352,22,19,4,497,162,170,1057,9576.000
1052,2019,Mitte,10113,Alexanderplatz,173,102,1966,500,420,11233,63,587,940,137,43,12,1307,381,1133,3813,9576.000
1053,2019,Mitte,10114,Brunnenstraße Süd,40,29,268,64,79,1859,39,182,361,64,18,7,424,172,86,902,9576.000
1054,2019,Mitte,10221,Moabit West,66,29,685,210,202,2107,47,322,326,93,28,15,641,91,618,1409,10.427


### Sanitizing NaNs

In [132]:
crimes2019_density[crimes2019_density['Density'].isnull()]

,Year,District,Code,Location,Robbery,Street_robbery,Injury,Agg_assault,Threat,Theft,Car,From_car,Bike,Burglary,Fire,Arson,Damage,Graffiti,Drugs,Local,Density
1060,2019,Mitte,19900,"Bezirk (Mi), nicht zuzuordnen",5,4,19,7,5,94,0,6,10,0,0,0,15,7,17,34,NaN
1063,2019,Friedrichshain-Kreuzberg,20303,nördliche Luisenstadt,94,67,524,215,99,2509,27,238,300,48,26,8,407,84,493,1103,NaN
1064,2019,Friedrichshain-Kreuzberg,20304,südliche Luisenstadt,182,150,530,251,125,2343,37,219,440,52,44,15,519,104,1310,1316,NaN
1065,2019,Friedrichshain-Kreuzberg,20405,Karl-Marx-Allee-Nord,38,26,225,58,61,1260,50,132,222,45,8,5,298,59,58,634,NaN
1066,2019,Friedrichshain-Kreuzberg,20407,Karl-Marx-Allee-Süd,98,74,787,281,160,3618,36,215,334,33,20,9,480,120,770,1573,NaN
1067,2019,Friedrichshain-Kreuzberg,20506,Frankfurter Allee Nord,30,16,376,109,96,1839,33,84,353,49,48,9,604,118,121,1212,NaN
1068,2019,Friedrichshain-Kreuzberg,20508,Frankfurter Allee Süd FK,132,92,972,353,168,4273,48,259,775,68,39,15,1062,335,1322,2606,NaN
1069,2019,Friedrichshain-Kreuzberg,29900,"Bezirk (Fh-Kb), nicht zuzuordnen",2,2,4,2,2,38,0,1,5,0,1,1,3,1,9,10,NaN
1086,2019,Pankow,39900,"Bezirk (Pk), nicht zuzuordnen",1,1,11,3,1,53,1,8,10,0,1,0,8,2,13,18,NaN
1104,2019,Charlottenburg-Wilmersdorf,49900,"Bezirk (Ch-Wi), nicht zuzuordnen",1,1,3,2,7,40,1,8,9,0,2,2,4,0,1,11,NaN


From the list above, it will deleted the elemens in the coloum "Location" that containt "nicht zuzuordnen" -> not assignable

In [134]:
crimes2019_density = crimes2019_density[~crimes2019_density.Location.str.contains("nicht zuzuordne")]
crimes2019_density.shape

(138, 21)

#### Now lets see how many NaN remains in the data

In [135]:
crimes2019_density[crimes2019_density['Density'].isnull()]

,Year,District,Code,Location,Robbery,Street_robbery,Injury,Agg_assault,Threat,Theft,Car,From_car,Bike,Burglary,Fire,Arson,Damage,Graffiti,Drugs,Local,Density
1063,2019,Friedrichshain-Kreuzberg,20303,nördliche Luisenstadt,94,67,524,215,99,2509,27,238,300,48,26,8,407,84,493,1103,NaN
1064,2019,Friedrichshain-Kreuzberg,20304,südliche Luisenstadt,182,150,530,251,125,2343,37,219,440,52,44,15,519,104,1310,1316,NaN
1065,2019,Friedrichshain-Kreuzberg,20405,Karl-Marx-Allee-Nord,38,26,225,58,61,1260,50,132,222,45,8,5,298,59,58,634,NaN
1066,2019,Friedrichshain-Kreuzberg,20407,Karl-Marx-Allee-Süd,98,74,787,281,160,3618,36,215,334,33,20,9,480,120,770,1573,NaN
1067,2019,Friedrichshain-Kreuzberg,20506,Frankfurter Allee Nord,30,16,376,109,96,1839,33,84,353,49,48,9,604,118,121,1212,NaN
1068,2019,Friedrichshain-Kreuzberg,20508,Frankfurter Allee Süd FK,132,92,972,353,168,4273,48,259,775,68,39,15,1062,335,1322,2606,NaN
1130,2019,Tempelhof-Schöneberg,70707,Lichtenrade,33,22,329,54,129,1008,80,225,126,95,36,12,398,71,69,675,NaN
1185,2019,Lichtenberg,110512,Rummelsburger Bucht,4,3,58,14,20,332,11,55,99,12,9,6,80,28,11,147,NaN
1186,2019,Lichtenberg,110513,Karlshorst,6,6,117,26,49,720,38,112,171,73,8,4,131,24,13,288,NaN
1190,2019,Reinickendorf,122141,West 1 - Tegel-Süd/Flughafensee,9,4,256,69,109,1233,21,217,43,37,9,3,142,6,210,528,NaN


#### It is possible to see that the "Code" does not match the list for Codes in the population table. However the name of the Distric can be matched with the coloum "Bezirk"
#### I would prefere to keep this values, rather than delete them. 
#### I will make a mean of the "Bezirk" and assing the mean to each of the NaN inthe table crimes2019_density

In [165]:
for i, c in crimes2019_density[crimes2019_density['Density'].isnull()].iterrows():
    density= population.loc[population['Bezirk'] == c['District']].groupby(['Bezirk'])['Einwohnerpro km²'].mean().item()
    crimes2019_density.at[i,'Density'] = density

In [183]:
print('NaN elements in consolodidated table: ',crimes2019_density[crimes2019_density['Density'].isnull()].shape[0])
print('total number of elements: ',crimes2019_density.shape[0])

NaN elements in consolodidated table:  0
total number of elements:  138


In [184]:
crimes2019_density.head()

,Year,District,Code,Location,Robbery,Street_robbery,Injury,Agg_assault,Threat,Theft,Car,From_car,Bike,Burglary,Fire,Arson,Damage,Graffiti,Drugs,Local,Density
1050,2019,Mitte,10111,Tiergarten Süd,60,35,365,92,128,2271,15,198,296,55,13,6,347,77,231,980,9576.000
1051,2019,Mitte,10112,Regierungsviertel,42,20,554,136,152,3692,13,172,352,22,19,4,497,162,170,1057,9576.000
1052,2019,Mitte,10113,Alexanderplatz,173,102,1966,500,420,11233,63,587,940,137,43,12,1307,381,1133,3813,9576.000
1053,2019,Mitte,10114,Brunnenstraße Süd,40,29,268,64,79,1859,39,182,361,64,18,7,424,172,86,902,9576.000
1054,2019,Mitte,10221,Moabit West,66,29,685,210,202,2107,47,322,326,93,28,15,641,91,618,1409,10.427


## Adding venues to the table